In [1]:
import math
from collections import namedtuple
import os
import re
import datetime as dt
import json

import numpy as np
import matplotlib

matplotlib.use("QT5Agg")

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.collections import PathCollection

from PyQt5 import QtWidgets

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar

from pprint import pprint

# %matplotlib qt5

### Define some helper methods and data structures

In [2]:
GeoExtent = namedtuple('GeoExtent', ['x_min', 'y_max', 'x_max', 'y_min'])
GeoAffine = namedtuple('GeoAffine', ['ul_x', 'x_res', 'rot_1', 'ul_y', 'rot_2', 'y_res'])
GeoCoordinate = namedtuple('GeoCoordinate', ['x', 'y'])
RowColumn = namedtuple('RowColumn', ['row', 'column'])
RowColumnExtent = namedtuple('RowColumnExtent', ['start_row', 'start_col', 'end_row', 'end_col'])
CONUS_EXTENT = GeoExtent(x_min=-2565585,
                         y_min=14805,
                         x_max=2384415,
                         y_max=3314805)
bands = ('blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'thermal')
indices = ('ndvi', 'msavi', 'evi', 'savi', 'ndmi', 'nbr', 'nbr2')
band_info = {b: {'coefs': [], 'inter': [], 'pred': []} for b in bands}

### Required parameters manually entered here

In [3]:
JSON_DIR = r'Z:\sites\chesapeake\pyccd-results\H28V09\2017.08.18\json'

CACHE_DIR = r'Z:\sites\chesapeake\ARD\h28v09\cache'

arc_paste = '1772368 1827300'

H = 28

V = 9

### Generate file inventory lists

In [4]:
JSON_INV = [os.path.join(JSON_DIR, f) for f in os.listdir(JSON_DIR)]

CACHE_INV = [os.path.join(CACHE_DIR, f) for f in os.listdir(CACHE_DIR)]

### Set time parameters

In [5]:
# These values are hardcoded for now
# May want to switch to determining these from the PyCCD results or ARD obs.
BEGIN_DATE = dt.date(year=1982, month=1, day=1)
END_DATE = dt.date(year=2015, month=12, day=31)

### Define all functions

In [6]:
def geospatial_hv(h, v, loc):
    """
    Geospatial extent and 30m affine for a given ARD grid location.
    """
    xmin = loc.x_min + h * 5000 * 30
    xmax = loc.x_min + h * 5000 * 30 + 5000 * 30
    ymax = loc.y_max - v * 5000 * 30
    ymin = loc.y_max - v * 5000 * 30 - 5000 * 30

    return (GeoExtent(x_min=xmin, x_max=xmax, y_max=ymax, y_min=ymin),
            GeoAffine(ul_x=xmin, x_res=30, rot_1=0, ul_y=ymax, rot_2=0, y_res=-30))

In [7]:
def geo_to_rowcol(affine, coord):
    """
    Transform geo-coordinate to row/col given a reference affine.
    
    Yline = (Ygeo - GT(3) - Xpixel*GT(4)) / GT(5)
    Xpixel = (Xgeo - GT(0) - Yline*GT(2)) / GT(1)
    """
    row = (coord.y - affine.ul_y - affine.ul_x * affine.rot_2) / affine.y_res
    col = (coord.x - affine.ul_x - affine.ul_y * affine.rot_1) / affine.x_res

    return RowColumn(row=int(row),
                     column=int(col))

In [8]:
def rowcol_to_geo(affine, rowcol):
    """
    Transform a row/col into a geospatial coordinate given reference affine.
    
    Xgeo = GT(0) + Xpixel*GT(1) + Yline*GT(2)
    Ygeo = GT(3) + Xpixel*GT(4) + Yline*GT(5)
    """
    x = affine.ul_x + rowcol.column * affine.x_res + rowcol.row * affine.rot_1
    y = affine.ul_y + rowcol.column * affine.rot_2 + rowcol.row * affine.y_res

    return GeoCoordinate(x=x, y=y)

In [9]:
def load_cache(file):
    """
    Load the cache file and split the data into the image IDs and values
    """
    data = np.load(file)
    return data['Y'], data['image_IDs']

In [10]:
def find_file(file_ls, string):
    """
    Return the first str in a list of strings that contains string.
    """
    gen = filter(lambda x: string in x, file_ls)
    return next(gen, None)

In [11]:
def imageid_date(image_ids):
    """
    Extract the ordinal day from the ARD image name.
    """
    return np.array([dt.datetime.strptime(d[15:23], '%Y%m%d').toordinal()
                     for d in image_ids])

In [12]:
def mask_daterange(dates):
    """
    Create a mask for values outside of the global BEGIN_DATE and END_DATE.
    :param dates:
    :return:
    """
    return np.logical_and(dates >= BEGIN_DATE.toordinal(), dates < END_DATE.toordinal())

In [13]:
def find_chipcurve(results_chip, coord):
    """
    Find the results for the specified coordinate.
    """
    with open(results_chip, 'r') as f:
        results = json.load(f)
    
    gen = filter(lambda x: coord.x == x['x'] and coord.y == x['y'], results)
    
    return next(gen, None)

In [14]:
def extract_cachepoint(coord):
    """
    Extract the spectral values from the cache file.
    """

    rowcol = geo_to_rowcol(PIXEL_AFFINE, coord)
    
    data, image_ids = load_cache(find_file(CACHE_INV, 'r{}'.format(rowcol.row)))
    
    dates = imageid_date(image_ids)
    
    return image_ids, data[:, :, rowcol.column], dates
   

In [15]:
def extract_jsoncurve(coord):
        """
        Extract the pyccd information from the json file representing a chip of results.
        """
        pixel_rowcol = geo_to_rowcol(PIXEL_AFFINE, coord)
        pixel_coord = rowcol_to_geo(PIXEL_AFFINE, pixel_rowcol)

        chip_rowcol = geo_to_rowcol(CHIP_AFFINE, coord)
        chip_coord = rowcol_to_geo(CHIP_AFFINE, chip_rowcol)

        file = find_file(JSON_INV,
                              "H{:02d}V{:02d}_{}_{}.json".format(H, V, chip_coord.x, chip_coord.y))
        result = find_chipcurve(file, pixel_coord)

        return json.loads(result["result"])

In [16]:
def predicts(days, coef, intercept):
    return (intercept + coef[0] * days +
            coef[1]*np.cos(days*1*2*np.pi/365.25) + coef[2]*np.sin(days*1*2*np.pi/365.25) +
            coef[3]*np.cos(days*2*2*np.pi/365.25) + coef[4]*np.sin(days*2*2*np.pi/365.25) +
            coef[5]*np.cos(days*3*2*np.pi/365.25) + coef[6]*np.sin(days*3*2*np.pi/365.25))

In [17]:
def arcpaste_to_coord(string):
    pieces = string.split()
    
    return GeoCoordinate(x=float(re.sub(',', '', pieces[0])),
                         y=float(re.sub(',', '', pieces[1])))

In [18]:
def test_data(end_date, dates_in, dates_out, data, dates, date_mask, ccd_mask):
    """
    Test the dates for the presence of duplicates, and compare both with and without duplicate counts to the number
    of observations in the PyCCD internal processing mask.  One possible source of duplicates is equal date
    observations from Landsat 7 and 8.  During Landsat 8's ascension into orbit there was a brief period of time
    where the two sensors 'overlapped' to allow instrument calibration, so duplicate acquisitions are possible but
    shouldn't be present because these particular Landsat 8 observations should have been removed from the ARD
    source directory.
    Another potential source of duplicate observations (i.e. dates) is when the ARD is re-ingested, it's scene ID
    may contain a different access date.  For example:
    LE07_CU_013005_20041223_20170731_C01_V01
    LE07_CU_013005_20041223_20170801_C01_V01
    These folders contain the same Landsat 7 observation acquired on 2004-12-23 but they have different accessed
    dates which is why one didn't overwrite the other.

    :return:
    """
    # TODO return either the second or first of duplicate pairs, need to figure out which (does it matter?)

    if len(dates_in) == len(ccd_mask):

        print("The number of observations is consistent with the length of the PyCCD internal processing mask.\n"
              "No changes to the input observations are necessary.")

        return end_date, dates_in, dates_out, data, date_mask, ccd_mask

    if len(np.unique(dates_in)) != len(dates_in) and len(np.unique(dates_in)) == len(ccd_mask):

        print("There is a duplicate date occurrence in observations.  Removing duplicate occurrences makes the "
              "number of observations consistent with the length of the PyCCD internal processing mask.")

        dupes = [item for item, count in Counter(dates).items() if count > 1]

        dates, ind, counts = np.unique(dates, return_index=True, return_counts=True)

        print(f"Duplicate dates: \n\t{[dt.datetime.fromordinal(d) for d in dupes]}")

        data = data[:, ind]

        date_mask = mask_daterange(dates)

        dates_in = dates[date_mask]

        dates_out = dates[~date_mask]

        return end_date, dates_in, dates_out, data, date_mask, ccd_mask

    if len(dates_in) != len(ccd_mask) and len(np.unique(dates_in)) != len(ccd_mask):

        # TODO Must check the date mask range, some tiles are inclusive of the end date
        end_date = dt.date(year=2016, month=1, day=1)

        date_mask = mask_daterange(dates)

        dates_in = dates[date_mask]
        dates_out = dates[~date_mask]

        # Try using the inclusive date mask
        if len(dates_in) == len(ccd_mask):
            print(
                "The number of observations is consistent with the length of the PyCCD internal processing mask.\n"
                "No changes to the input observations are necessary.")

            return end_date, dates_in, dates_out, data, date_mask, ccd_mask

        # If the inclusive date mask doesn't match the processing mask, then resort to using the PIXELQA
        else:
            print("There is an inconsistency with the length of the processing mask, therefore it will not be used."
                "  PIXELQA band will be used to filter observations.")
            ccd_mask = get_pqa_mask()

            return end_date, dates_in, dates_out, data, date_mask, ccd_mask

In [19]:
def get_predicts(num):
    """
    Return the list of model prediction values in the time series for a particular band or bands

    :param num: int or list
    :return: list
    """

    # Check for type int, create list if true
    if isinstance(num, int):
        num = [num]

    return [predicted_values[m * len(bands) + n] for n in num
            for m in range(len(results["change_models"]))]

In [20]:
def get_pqa_mask(qa, date_mask):
    """
    Generate a mask from the Pixel QA
    :param qa:
    :param date_mask:
    :return:
    """
    clr_vals = [66, 68, 322, 324]

    pixelqa_in = qa[date_mask]

    pixelqa_mask = np.zeros_like(pixelqa_in, dtype=np.bool)

    for val in clr_vals:
        pixelqa_mask[pixelqa_in == val] = True

    return pixelqa_mask

In [21]:
def msavi(R, NIR):
    # Modified Soil Adjusted Vegetation Index
    
    return (2.0 * NIR + 1.0 - ((2.0 * NIR + 1.0)**2.0 - 8.0 * (NIR - R))**0.5) / 2.0

def ndvi(R, NIR):
    # Normalized Difference Vegetation Index
    
    return (NIR - R) / (NIR + R)

def evi(B, R, NIR, G=2.5, L=1.0, C1=6, C2=7.5):
    # Enhanced Vegetation Index
    
    return G * ((NIR - R) / (NIR + C1 * R - C2 * B + L))

def savi(R, NIR, L=0.5):
    # Soil Adjusted Vegetation Index
    
    return ((NIR - R) / (NIR + R + L)) * (1 + L)

def ndmi(NIR, SWIR1):
    # Normalized Difference Moisture Index
    
    return (NIR - SWIR1) / (NIR + SWIR1)

def nbr(NIR, SWIR2):
    # Normalized Burn Ratio
    
    return (NIR - SWIR2) / (NIR + SWIR2)

def nbr2(SWIR1, SWIR2):
    # Normalized Burn Ratio 2
    
    return (SWIR1 - SWIR2) / (SWIR1 + SWIR2)  
    

### Produce necessary geo information

In [22]:
coord = arcpaste_to_coord(arc_paste)

EXTENT, PIXEL_AFFINE = geospatial_hv(H, V, CONUS_EXTENT)

CHIP_AFFINE = GeoAffine(ul_x=PIXEL_AFFINE.ul_x, x_res=3000, rot_1=0, ul_y=PIXEL_AFFINE.ul_y, rot_2=0, y_res=-3000)

### Load results from the json file

In [23]:
results = extract_jsoncurve(coord)

### Load observation information from the cache file

In [24]:
imageIDs, data, dates = extract_cachepoint(coord)

### Generate the date mask

In [25]:
date_mask = mask_daterange(dates)

### Generate an array containing the results' internal processing mask

In [26]:
ccd_mask = np.array(results["processing_mask"], dtype=bool)

### Create lists of dates in/out of the BEGIN-to-END date range

In [27]:
dates_in = dates[date_mask]
dates_out = dates[~date_mask]

### Create an array of the PIXELQA

In [28]:
qa = data[-1]

### Test the length of the observed values against the length of the processing mask

In [29]:
END_DATE, dates_in, dates_out, data, date_mask, ccd_mask = test_data(end_date=END_DATE, dates_in=dates_in, 
                                                                     dates_out=dates_out, data=data, dates=dates, 
                                                                     date_mask=date_mask, ccd_mask=ccd_mask)

The number of observations is consistent with the length of the PyCCD internal processing mask.
No changes to the input observations are necessary.


### Generate mask based on the PIXELQA fill value

In [30]:
fill_mask = np.ones_like(qa, dtype=np.bool)
fill_mask[qa == 1] = False

### Get the PIXELQA masks in/out of the BEGIN-END date range

In [31]:
fill_in = fill_mask[date_mask]
fill_out = fill_mask[~date_mask]

### Create a total mask by combining the internal mask and the PIXELQA-in-date-range mask

In [32]:
total_mask = np.logical_and(ccd_mask, fill_in)

### Rescale the brightness temperature to match the predicted values

In [33]:
temp_thermal_data = np.copy(data[6])
temp_thermal_data[fill_mask] = temp_thermal_data[fill_mask] * 10 - 27315
data[6] = np.copy(temp_thermal_data)

### Create containers that will store lists of values for the plots

In [34]:
predicted_values = []
prediction_dates = []
break_dates = []
start_dates = []
end_dates = []

### Retrieve and display information from the PyCCD results for plotting

In [35]:
for num, result in enumerate(results['change_models']):
    print('Result: {}'.format(num))
    print('Start Date: {}'.format(dt.date.fromordinal(result['start_day'])))
    print('End Date: {}'.format(dt.date.fromordinal(result['end_day'])))
    print('Break Date: {}'.format(dt.date.fromordinal(result['break_day'])))
    print('QA: {}'.format(result['curve_qa']))
    print('Change prob: {}'.format(result['change_probability']))
    
    days = np.arange(result['start_day'], result['end_day'] + 1)
    
    break_dates.append(result['break_day'])
    start_dates.append(result['start_day'])
    end_dates.append(result['end_day'])
    
    for b in bands:
        band_info[b]['inter'] = result[b]['intercept']
        band_info[b]['coefs'] = result[b]['coefficients']
        
        band_info[b]['pred'] = predicts(days, result[b]['coefficients'], result[b]['intercept'])
        
        prediction_dates.append(days)
        predicted_values.append(band_info[b]['pred'])

Result: 0
Start Date: 1982-11-11
End Date: 2014-11-03
Break Date: 2014-11-03
QA: 8
Change prob: 0


### Calculate indices from observed values

In [36]:
EVI = evi(B=data[0], NIR=data[3], R=data[2])

NDVI = ndvi(R=data[2], NIR=data[3])

MSAVI = msavi(R=data[2], NIR=data[3])

SAVI = savi(R=data[2], NIR=data[3])

NDMI = ndmi(NIR=data[3], SWIR1=data[4])

NBR = nbr(NIR=data[3], SWIR2=data[5])

NBR2 = nbr2(SWIR1=data[4], SWIR2=data[5])

C:\Users\dzelenak\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in true_divide


### Calculate indices from the results' change models

In [37]:
# The change models are stored by order of model, then band number.  For example, the band values for the first change 
# model are represented by indices 0-5, the second model by indices 6-11, and so on.

NDVI_ = [ndvi(NIR=predicted_values[m * len(bands) + 3],
                                  R=predicted_values[m * len(bands) + 2])
              for m in range(len(results["change_models"]))]

MSAVI_ = [msavi(R=predicted_values[m * len(bands) + 2],
                                    NIR=predicted_values[m * len(bands) + 3])
               for m in range(len(results["change_models"]))]

EVI_ = [evi(B=predicted_values[m * len(bands)],
                                NIR=predicted_values[m * len(bands) + 3],
                                R=predicted_values[m * len(bands) + 2])
             for m in range(len(results["change_models"]))]

SAVI_ = [savi(NIR=predicted_values[m * len(bands) + 3],
                                  R=predicted_values[m * len(bands) + 2])
              for m in range(len(results["change_models"]))]

NDMI_ = [ndmi(NIR=predicted_values[m * len(bands) + 3],
                                  SWIR1=predicted_values[m * len(bands) + 4])
              for m in range(len(results["change_models"]))]

NBR_ = [nbr(NIR=predicted_values[m * len(bands) + 3],
                                SWIR2=predicted_values[m * len(bands) + 5])
             for m in range(len(results["change_models"]))]

NBR2_ = [nbr2(SWIR1=predicted_values[m * len(bands) + 4],
                                  SWIR2=predicted_values[m * len(bands) + 5])
              for m in range(len(results["change_models"]))]

### Create some data structures

In [38]:
index_lookup = {"NDVI": (NDVI, NDVI_),
                 "MSAVI": (MSAVI, MSAVI_),
                 "EVI": (EVI, EVI_),
                 "SAVI": (SAVI, SAVI_),
                 "NDMI": (NDMI, NDMI_),
                 "NBR": (NBR, NBR_),
                 "NBR-2": (NBR2, NBR2_)}

band_lookup = {"Blue": (data[0], get_predicts(0)),
                "Green": (data[1], get_predicts(1)),
                "Red": (data[2], get_predicts(2)),
                "NIR": (data[3], get_predicts(3)),
                "SWIR-1": (data[4], get_predicts(4)),
                "SWIR-2": (data[5], get_predicts(5)),
                "Thermal": (data[6], get_predicts(6))}

# Combine these two dictionaries
all_lookup = {**band_lookup, **index_lookup}

### Select which dictionary to use for plotting

In [39]:
plot_data = all_lookup

### Set up some plot features

In [40]:
plt.style.use('ggplot')

# get year values for labeling plots
year1 = str(dt.datetime.fromordinal(dates[0]))[:4]
year2 = str(dt.datetime.fromordinal(dates[-1]))[:4]

# List every-other year
years = range(int(year1), int(year2) + 2, 2)

# List every-single year
years_ = range(int(year1), int(year2) + 2, 1)

# list of datetime objects with YYYY-MM-dd pattern using July 1 for month and day
t = [dt.datetime(yx, 7, 1) for yx in years]

# list of datetime objects with YYYY-MM-dd pattern using January 1 for month and day
t_ = [dt.datetime(yx, 1, 1) for yx in years_]

# list of ordinal time objects
ord_time = [dt.datetime.toordinal(tx) for tx in t]

# list of datetime formatted strings
x_labels = [str(dt.datetime.fromordinal(int(L)))[:10] if L != "0.0" and L != "" else "0" for L in ord_time]

# decide whether or not to draw masked values
masked_on = True

# decide whether or not to draw model values
model_on = True

### Specify inidividual bands here if desired

In [41]:
all_keys = plot_data.keys()
all_keys

dict_keys(['Blue', 'Green', 'Red', 'NIR', 'SWIR-1', 'SWIR-2', 'Thermal', 'NDVI', 'MSAVI', 'EVI', 'SAVI', 'NDMI', 'NBR', 'NBR-2'])

In [42]:
keys = ["Blue"]

### Define a function for clicking on the plot to select the nearest artist with a set picker

In [43]:
# https://matplotlib.org/examples/event_handling/pick_event_demo.html
def line_picker(line, mouseevent):
    """
    find the points within a certain distance from the mouseclick in
    data coords and attach some extra attributes, pickx and picky
    which are the data points that were picked
    """
    if mouseevent.xdata is None:
        return False, dict()
    
    xdata = line.get_xdata()
    ydata = line.get_ydata()
    
    maxd = 1.0
    d = np.sqrt((xdata - mouseevent.xdata)**2. + (ydata - mouseevent.ydata)**2.)

    ind = np.nonzero(np.less_equal(d, maxd))
    
    if len(ind):
        pickx = np.take(xdata, ind)
        picky = np.take(ydata, ind)
        props = dict(ind=ind, pickx=pickx, picky=picky)
        return True, props
    
    else:
        return False, dict()

### Create a (currently empty) mapping of data series to artists

In [44]:
artist_map = {}

### Render the artist primitives and containers

In [45]:
# Create the artist containers
fig, axes = plt.subplots(nrows=len(keys), ncols=1, figsize=(18, len(keys) * 6), dpi=65, squeeze=False)

for num, b in enumerate(keys):
    # Plot the observations within the PyCCD time range
    points1 = axes[num, 0].scatter(x=dates_in[total_mask], y=plot_data[b][0][date_mask][total_mask], s=44, c="green", marker='o', edgecolors="k",
                         label="Obs.", picker=5)
    
    artist_map[points1] = [dates_in[total_mask], plot_data[b][0][date_mask][total_mask]]
    
    # Plot the observations outside of the PyCCD time range
    points2 = axes[num, 0].scatter(x=dates_out[fill_out], y=plot_data[b][0][~date_mask][fill_out], s=21, c='red', marker='o', edgecolors='k', 
                      label="Unused", picker=5)
    
    artist_map[points2] = [dates_out[fill_out], plot_data[b][0][~date_mask][fill_out]]
    
    # Plot the observed values masked out by PyCCD
    if masked_on is True:
        
        #Remove the 0-valued masked obserations for the index plots
        if b in keys:
            index_plot = plot_data[b][0][date_mask][~ccd_mask]
            
            points3 = axes[num, 0].scatter(x=dates_in[~ccd_mask][index_plot != 0], y=index_plot[index_plot != 0], s=21, c="0.65",
                             marker="o", label="Masked", picker=5)
            
            artist_map[points3] = [dates_in[~ccd_mask][index_plot != 0], index_plot[index_plot != 0]]
            
        else:
            points3 = axes[num, 0].scatter(x=dates_in[~ccd_mask], y=plot_data[b][0][date_mask][~ccd_mask], s=21, c="0.65", 
                             marker="o", label="Masked", picker=5)
            
            artist_map[points3] = [dates_in[~ccd_mask], plot_data[b][0][date_mask][~ccd_mask]]
            
    # Give each subplot a title
    axes[num, 0].set_title(f"{b}")
    
    # Plot the model start, end, and break days
    if model_on is True:
        # Get list of instances where break=start date
        match_dates = [b for b in break_dates for s in start_dates if b==s]
    
        for ind, e in enumerate(end_dates):
            if ind == 0:
                axes[num, 0].axvline(e, color="maroon", linewidth=1.5, label="End", picker=line_picker)
                
            else:
                # Plot without a label to remove duplicates in the legend
                axes[num, 0].axvline(e, color="maroon", linewidth=1.5, picker=None)
                
        for ind, br in enumerate(break_dates):
            if ind == 0:
                axes[num, 0].axvline(br, color="r", linewidth=1.5, label="Break", picker=line_picker)
                
            else:
                axes[num, 0].axvline(br, color="r", linewidth=1.5, picker=line_picker)
                
        for ind, s in enumerate(start_dates):
            if ind == 0:
                axes[num, 0].axvline(s, color="b", linewidth=1.5, label="Start", picker=line_picker)
                
            else:
                axes[num, 0].axvline(s, color="b", linewidth=1.5, picker=line_picker)
                
        for ind, m in enumerate(match_dates):
            if ind == 0:
                axes[num, 0].axvline(m, color="magenta", linewidth=1, label="Break=Start", picker=line_picker)
                
            else:
                axes[num, 0].axvline(m, color="magenta", linewidth=1, picker=line_picker)
                
        # Predicted Curves
        for c in range(0, len(results["change_models"])):
            if c == 0:
                axes[num, 0].plot(prediction_dates[c * len(bands)], plot_data[b][1][c], "orange",
                                 linewidth=2, alpha=0.8, label="Model Fit", picker=line_picker)
                
            else:
                axes[num, 0].plot(prediction_dates[c * len(bands)], plot_data[b][1][c], "orange",
                                 linewidth=2, alpha=0.8, picker=line_picker)
                
    # Get ymin and ymax values to constrain the plot window size
    if b in index_lookup.keys():
        ymin = min(plot_data[b][0][date_mask][total_mask]) - 0.15
        ymax = max(plot_data[b][0][date_mask][total_mask]) + 0.1

    else:
        ymin = min(plot_data[b][0][date_mask][total_mask]) - 700
        ymax = max(plot_data[b][0][date_mask][total_mask]) + 500

    axes[num, 0].set_ylim([ymin, ymax])

    # Add x-ticks and x-tick_labels
    # axes[num, 0].set_xticks(ord_time)

    # axes[num, 0].set_xticklabels(x_labels, rotation=70, horizontalalignment="right")

    # Display the x and y values where the cursor is placed on a subplot
    axes[num, 0].format_coord = lambda x, y: "({0:f}, ".format(y) +  \
                                             "{0:%Y-%m-%d})".format(dt.datetime.fromordinal(int(x)))

    # Plot a vertical line at January 1 of each year on the time series
    for y in t_:
        axes[num, 0].axvline(y, color="dimgray", linewidth=1.5, picker=None)

    # Only add a legend to the first subplot to avoid repetition and wasted space
    if b == list(keys)[0]:
        axes[num, 0].legend(ncol=1, loc="upper left", bbox_to_anchor=(1.0, 1.0),
                            borderaxespad=0.)
    
fig.tight_layout()

fig.subplots_adjust(right=0.9)

In [46]:
class MplCanvas(FigureCanvas):
    def __init__(self, fig):
        self.fig = fig
        
        FigureCanvas.__init__(self, self.fig)
        
        if len(fig.axes) >= 3:
            sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Ignored, QtWidgets.QSizePolicy.Minimum)
        
        else:
            sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Ignored, QtWidgets.QSizePolicy.Ignored)
        
        sizePolicy.setHorizontalStretch(1)
        sizePolicy.setVerticalStretch(1)
        
        FigureCanvas.setSizePolicy(self, sizePolicy)
        
        FigureCanvas.updateGeometry(self)

In [47]:
# https://stackoverflow.com/questions/42622146/scrollbar-on-matplotlib-showing-page
class PlotWindow(QtWidgets.QMainWindow):
    def __init__(self, fig):
        self.qapp = QtWidgets.QApplication([])
        
        QtWidgets.QMainWindow.__init__(self)
        
        self.widget = QtWidgets.QWidget()
        self.setCentralWidget(self.widget)
        self.widget.setLayout(QtWidgets.QVBoxLayout())
        self.widget.layout().setContentsMargins(0,0,0,0)
        self.widget.layout().setSpacing(0)
        
        self.fig = fig
        self.canvas = MplCanvas(self.fig)
        self.canvas.draw()
            
        
        # def line_pick(event):
            # event.pickx = the x-data value at the nearset point
            # event.picky = the y-data value at the nearest point
            # print('onpick line:', dt.datetime.fromordinal(int(event.pickx)), event.picky)
            
        def point_pick(event):
            
            mouseevent = event.mouseevent
            
            artist = event.artist
            
            # Only works using left-click (event.mouseevent.button==1) 
            # and on any of the scatter point series (PathCollection artists)
            if isinstance(artist, PathCollection) and mouseevent.button==1:
            
                ind = event.ind
                # ax = event.inaxes
                # ax = fig.gca()
                x=artist_map[artist][0]
                y=artist_map[artist][1]
                print(f'clicked button: {mouseevent.button} \n {dt.datetime.fromordinal(int(mouseevent.xdata)), mouseevent.ydata}\n \
                      ind={ind}\n artist={artist}\n date(x)={dt.datetime.fromordinal(int(np.take(x, ind)))}\n \
                      val(y)={np.take(y, ind)}')
            
        
        self.nav = NavigationToolbar(self.canvas, self.widget)
        
        self.widget.layout().addWidget(self.nav)
        
        self.widget.layout().addWidget(self.canvas)
        
        self.scroll = QtWidgets.QScrollArea(self.widget)
        self.scroll.setWidgetResizable(True)
        
        self.scroll.setWidget(self.canvas)
               
        
        self.widget.layout().addWidget(self.scroll)
        
        self.canvas.mpl_connect("pick_event", point_pick)
        
        # self.canvas.mpl_connect("pick_event", line_pick)
                        
        self.show()
        exit(self.qapp.exec_())

In [48]:
# Display the figure
look = PlotWindow(fig)


clicked button: 1 
 (datetime.datetime(2017, 3, 17, 0, 0), 3865.717554958971)
                       ind=[47]
 artist=<matplotlib.collections.PathCollection object at 0x0000016B88974E80>
 date(x)=2017-03-08 00:00:00
                       val(y)=[3859]
clicked button: 1 
 (datetime.datetime(2017, 3, 17, 0, 0), 3865.717554958971)
                       ind=[47]
 artist=<matplotlib.collections.PathCollection object at 0x0000016B88974E80>
 date(x)=2017-03-08 00:00:00
                       val(y)=[3859]
clicked button: 1 
 (datetime.datetime(2017, 3, 17, 0, 0), 3865.717554958971)
                       ind=[47]
 artist=<matplotlib.collections.PathCollection object at 0x0000016B88974E80>
 date(x)=2017-03-08 00:00:00
                       val(y)=[3859]
clicked button: 1 
 (datetime.datetime(2016, 7, 7, 0, 0), 7754.7993591855811)
                       ind=[21]
 artist=<matplotlib.collections.PathCollection object at 0x0000016B88974E80>
 date(x)=2016-07-27 00:00:00
                       va

Some Notes:

It is desired to be able to select any series on the plot and have returned the x-coordinate (i.e. dt.datetime.fromordinal()) and the y-coordinate (reflectance or indices value).  Need to implement something in the code that allows to differentiate between the series because currently the reference is being made to a single date series range, and the return on clicking is not correct for any items in the series after the first index. - Done

Want to be able to draw a rectangle around groups of points and return their values as a .txt or .csv